In [1]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
db_user = ""
db_password = ""
db_host = ""  
db_port = "" 
db_name = ""
connection = f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection)

In [3]:
from datetime import datetime
ace_partenza = '08%' # tutta Emilia Romagna
ace_arrivo = '08|037|006%' # Bologna

intervallo_inizio = datetime(2019, 9, 15,  7,  0,  0)   # 2019-09-15 07:00:00
intervallo_fine   = datetime(2019, 9, 15, 9, 0,  0)   # 2019-09-15 09:30:00

limite = 10 # quanti flussi visualizzare

In [4]:
query = text("""
    SELECT
        CASE WHEN layerid < toid THEN layerid ELSE toid END AS ace1,
        CASE WHEN layerid < toid THEN toid ELSE layerid END AS ace2,
        SUM(datavalue) AS total_movements
    FROM movements
    WHERE (
        (layerid LIKE :ace_partenza) AND
        (toid LIKE :ace_arrivo ) AND
        (layerid NOT LIKE '08|037%')
    )
    AND datefrom >= :inizio
    AND dateto   <= :fine
    GROUP BY ace1, ace2
    ORDER BY total_movements DESC
    LIMIT :limite
""")

try:
    with engine.connect() as connection:
        chunks = pd.read_sql(query, connection, chunksize=1000, params = {
    "ace_arrivo": f"{ace_arrivo}%",
    "ace_partenza": f"{ace_partenza}%",
    "inizio": intervallo_inizio,
    "fine": intervallo_fine,
    "limite": limite
}) 
        df = pd.concat(chunks, ignore_index=True)  
    print(df)
except Exception as e:
    print(f"Errore durante l'esecuzione della query: {e}")

                 ace1                ace2  total_movements
0  08|037|006|001|000  08|040|012|000|000               25
1  08|036|006|000|000  08|037|006|001|000               22
2  08|037|006|001|000  08|099|001|000|000               22
3  08|037|006|003|003  08|039|014|001|000               19
4  08|037|006|007|071  08|099|001|000|000               19
5  08|037|006|001|000  08|039|007|000|001               19
6  08|037|006|001|000  08|099|013|000|002               18
7  08|035|033|002|000  08|037|006|001|000               16
8  08|036|023|002|000  08|037|006|001|000               15
9  08|037|006|001|000  08|039|014|002|004               15


In [5]:
# 1. Scarico solo le geometrie
query_geom =text("""SELECT layerid, geojson FROM geom_data""")
df_geom = pd.read_sql(query_geom, engine)

# 2. Rinomino per i join successivi
df_geom_from = df_geom.rename(columns={'layerid': 'ace1', 'geojson': 'geo1'})
df_geom_to = df_geom.rename(columns={'layerid': 'ace2', 'geojson': 'geo2'})

# 3. Faccio le due join in pandas
df = df.merge(df_geom_from, on='ace1', how='left')
df = df.merge(df_geom_to, on='ace2', how='left')
df

,ace1,ace2,total_movements,geo1,geo2
0,08|037|006|001|000,08|040|012|000|000,25,"{'id': '2b48479a-5a02-44b6-aeaf-b1b9c4aba69d',...","{'id': '902e2a1a-fc72-43f5-b5f5-203ff9db7a80',..."
1,08|036|006|000|000,08|037|006|001|000,22,"{'id': '2d6ef556-4293-467e-82f1-0cd79b6c7a40',...","{'id': '2b48479a-5a02-44b6-aeaf-b1b9c4aba69d',..."
2,08|037|006|001|000,08|099|001|000|000,22,"{'id': '2b48479a-5a02-44b6-aeaf-b1b9c4aba69d',...","{'id': '5b90e362-5bd4-425a-84a8-d7fec2a688fa',..."
3,08|037|006|003|003,08|039|014|001|000,19,"{'id': '285935bc-b278-4ad4-bae8-ba42b7fb03da',...","{'id': 'b192e047-3873-4003-8a52-9a2e568be68e',..."
4,08|037|006|007|071,08|099|001|000|000,19,"{'id': '6db59a42-ee90-41d0-9d1f-334bb1747742',...","{'id': '5b90e362-5bd4-425a-84a8-d7fec2a688fa',..."
5,08|037|006|001|000,08|039|007|000|001,19,"{'id': '2b48479a-5a02-44b6-aeaf-b1b9c4aba69d',...","{'id': 'd1764910-2923-4aaa-a64e-ce8d9fb56949',..."
6,08|037|006|001|000,08|099|013|000|002,18,"{'id': '2b48479a-5a02-44b6-aeaf-b1b9c4aba69d',...","{'id': '14b6a899-e430-4131-b11b-79d853ee8e56',..."
7,08|035|033|002|000,08|037|006|001|000,16,"{'id': '4f16c8dd-5a83-4e7c-87e6-b17caf8a711e',...","{'id': '2b48479a-5a02-44b6-aeaf-b1b9c4aba69d',..."
8,08|036|023|002|000,08|037|006|001|000,15,"{'id': '6eada483-9bc1-4509-96f0-d42ca2d25489',...","{'id': '2b48479a-5a02-44b6-aeaf-b1b9c4aba69d',..."
9,08|037|006|001|000,08|039|014|002|004,15,"{'id': '2b48479a-5a02-44b6-aeaf-b1b9c4aba69d',...","{'id': 'e23fcfed-53a4-4b50-b2d3-a88358d4f00b',..."


In [6]:
from shapely.geometry import shape
# 1. Crea dizionario layerid → (lat, lon)
geo_centroid_dict = {}

for _, row in df_geom.iterrows():
    try:
        geo = row['geojson']  # già dict!
        geom = shape(geo['geometry'])
        centroid = (geom.centroid.y, geom.centroid.x)
        geo_centroid_dict[row['layerid']] = centroid
    except Exception as e:
        print(f"Errore su {row['layerid']}: {e}")
        geo_centroid_dict[row['layerid']] = (None, None)

# 2. Crea nuove colonne facendo il mapping
df['lat1'] = df['ace1'].map(lambda x: geo_centroid_dict.get(x, (None, None))[0])
df['lon1'] = df['ace1'].map(lambda x: geo_centroid_dict.get(x, (None, None))[1])
df['lat2']   = df['ace2'].map(lambda x: geo_centroid_dict.get(x, (None, None))[0])
df['lon2']   = df['ace2'].map(lambda x: geo_centroid_dict.get(x, (None, None))[1])

In [7]:
import folium
from geopy.distance import geodesic
import branca.colormap as bcm

# Filtra righe valide con coordinate
df_plot = df.dropna(subset=['lat1', 'lon1', 'lat2', 'lon2'])

# Imposta valori min e max dal dataset (top 10)
#min_val = df_plot['total_movements'].min()
min_val = 0
#max_val = df_plot['total_movements'].max()
max_val = 100
# Colormap 
linear_colormap = bcm.LinearColormap(
    colors=['#fff5f0', '#fb6a4a', '#67000d'],
    vmin=min_val,
    vmax=max_val,
    caption='Totale spostamenti'
)

# Mappa base
mappa = folium.Map(
    location=[44.5, 11.3],
    zoom_start=8,
    tiles="CartoDB positron"
)

# Aggiunge la legenda
linear_colormap.add_to(mappa)

# Funzione per generare coordinate ad arco
def arc_coordinates(p1, p2, num_points=200, base_height=0.002):
    distance_km = geodesic(p1, p2).km
    height = min(0.05, base_height * distance_km)
    lats = np.linspace(p1[0], p2[0], num_points)
    lons = np.linspace(p1[1], p2[1], num_points)
    curve = height * np.sin(np.linspace(0, np.pi, num_points))
    lats += curve
    return list(zip(lats, lons))

# Aggiunge archi
for _, row in df_plot.iterrows():
    p1 = (row['lat1'], row['lon1'])
    p2 = (row['lat2'], row['lon2'])
    coords = arc_coordinates(p1, p2)
    value = row['total_movements']

    # Colore e peso dalla stessa colormap
    color = linear_colormap(value)
    norm_value = (value - min_val) / (max_val - min_val) if max_val > min_val else 0
    weight = 2 + 5 * norm_value  # Spessore ben visibile

    folium.PolyLine(
        locations=coords,
        weight=weight,
        color=color,
        opacity=0.6,
        tooltip=f"{row['ace1']} → {row['ace2']}: {value:,} spostamenti"
    ).add_to(mappa)

# Visualizza mappa
mappa